In [270]:
#Brker CNN arkitektur kombinert med PyTorch 


In [271]:
import numpy as np
import matplotlib.pyplot as plt
# import pandas as pd

#other libraries
# from tqdm import tqdm
# import time
# import random
import os
import sys
from pathlib import Path
import h5py

#torch specific
import torch
import torchvision as torchv
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch import Tensor
from torch.utils import data

from sklearn.model_selection import train_test_split

In [272]:
module_path = str(Path.cwd().parents[0].parents[0] / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

from dataloader import *
# from plotCreator import *

data_path0 = str(Path.cwd().parents[0].parents[0] / "data" / "bh" / "BH_n5_M8_res50_10000_events.h5")
data_path1 = str(Path.cwd().parents[0].parents[0] / "data" / "sph" / "PP13-Sphaleron-THR9-FRZ15-NB0-NSUBPALL_res50_10000_events.h5")

In [273]:
bhArray = dataToArray(data_path0)
sphArray = dataToArray(data_path1)

In [ ]:
dataArray = np.concatenate((bhArray,sphArray),axis=0)

In [ ]:
np.shape(dataArray)

In [ ]:
labelsArray = np.concatenate((np.zeros(10_000),np.ones(10_000)),axis=0)

In [ ]:
np.shape(labelsArray)

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

In [ ]:
trainData, testData, trainLabels, testLabels = train_test_split(dataArray, labelsArray, random_state=42)

In [ ]:
trainData = torch.from_numpy(trainData)
testData = torch.from_numpy(testData)
trainLabels = torch.from_numpy(trainLabels)
testLabels = torch.from_numpy(testLabels)

In [ ]:
train = torch.utils.data.TensorDataset(trainData, trainLabels)
test = torch.utils.data.TensorDataset(testData, testLabels)

In [ ]:
trainLoader = DataLoader(train, shuffle=True, batch_size=50)
testLoader = DataLoader(test, shuffle=True, batch_size=50)

In [ ]:
class CNNModel(nn.Module):
    def __init__(self, resolution, image_channels, num_classes, stride=1):
        super().__init__() # just run the init of parent class (nn.Module)
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=stride, padding_mode="circular", padding=3) # input is 3 channels, 32 output channels, 3x3 kernel / window
        #self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=stride)
        #F.conv2d(F.pad(input, pad=(5,5,5,5), mode='circular'), kernel, padding=0)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, stride=stride, padding_mode="circular", padding=3)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, stride=stride)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=stride, padding_mode="circular", padding=3) # input is 32, bc the first layer output 32. Then we say the output will be 64 channels, 5x5 kernel / window
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, stride=stride, padding_mode="circular", padding=3)
        self.conv5 = nn.Conv2d(64, 128, kernel_size=3, stride=stride, padding_mode="circular", padding=3)
        self.conv6 = nn.Conv2d(128, 128, kernel_size=3, stride=stride, padding_mode="circular", padding=3)

        self.max_pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.max_pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.max_pool3 = nn.MaxPool2d(kernel_size = 2, stride = 2)

        x = torch.randn(resolution, resolution, image_channels).view(-1, image_channels, resolution, resolution) #Dummy image used in to_linear func
        print(x.shape)
        self._to_linear = None
        self.convs(x)

        self.fc1 = nn.Linear(self._to_linear, 512) #flattening.
        self.fc2 = nn.Linear(512, num_classes) # 512 in, CLASSES out -> number of classes

    def convs(self, x:Tensor):
        # max pooling over 2x2
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.max_pool1(x)
        
        #x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = self.conv3(x)
        x= self.conv4(x)
        x = self.max_pool2(x)
        
        #x = F.max_pool2d(F.relu(self.conv4(x)), (2, 2))
        #x = self.conv5(x)
        #x = self.conv6(x)
        #x = self.max_pool3(x)

        #x = F.max_pool2d(F.relu(self.conv6(x)), (3, 3))

        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2] #calculates the output that needs to be flattened
        return x

    def forward(self, x:Tensor):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)  # .view is reshape ... this flattens X before 
        x = F.relu(self.fc1(x))
        x = self.fc2(x) # bc this is our output layer. No activation here.
        return F.softmax(x, dim=1)

In [ ]:
model = CNNModel()

In [ ]:
error = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
criterion = nn.NLLLoss()

In [ ]:
epochs = 20
steps = 0
running_loss = 0
print_every = 10
train_losses, test_losses = [], []
for epoch in range(epochs):
    for inputs, labels in trainLoader:

        steps += 1
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in testLoader:

                    inputs, labels = inputs.to(device),  labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    test_loss += batch_loss.item()
                    
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals =  top_class == labels.view(*top_class.shape)
                    accuracy +=   torch.mean(equals.type(torch.FloatTensor)).item()
            train_losses.append(running_loss/len(trainLoader))
            test_losses.append(test_loss/len(testLoader))                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testLoader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testLoader):.3f}")
            running_loss = 0
            model.train()